In [3]:
import pandas as pd
import psycopg2

class PostgresConnection:
    def __init__(self, dbname, user, password, host='localhost', port='5432'):
        self.dbname = dbname
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.conn = None
        self.cursor = None

    def connect(self):
        try:
            self.conn = psycopg2.connect(
                dbname=self.dbname,
                user=self.user,
                password=self.password,
                host=self.host,
                port=self.port
            )
            self.cursor = self.conn.cursor()
            print("Connected to PostgreSQL database!")
        except Exception as e:
            print(f"Error: {e}")

    def create_table(self):
        create_table_query = """
        CREATE TABLE IF NOT EXISTS telegram_post (
            id SERIAL PRIMARY KEY,
            date DATE NOT NULL,
            post_link TEXT NOT NULL,
            views TEXT,
            post_hour TIME,
            keyword TEXT,
            time_of_day TEXT
        );
        """
        try:
            self.cursor.execute(create_table_query)
            self.conn.commit()
            print("Table 'telegram_posts' created successfully!")
        except Exception as e:
            print(f"Error creating table: {e}")
            self.conn.rollback()

    def insert_data(self, data):
        if self.conn is not None:
            try:
                for row in data:
                    insert_query = """
                    INSERT INTO telegram_post (date, post_link, views, post_hour, keyword, time_of_day)
                    VALUES (%s, %s, %s, %s, %s, %s);
                    """
                    self.cursor.execute(insert_query, row)
                self.conn.commit()
                print("Data inserted successfully!")
            except Exception as e:
                print(f"Error inserting data: {e}")
                self.conn.rollback()

    def close_connection(self):
        if self.conn is not None:
            self.cursor.close()
            self.conn.close()
            print("Connection closed.")

def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

def extract_attributes(row):
    keywords = [
        '#CBE', '#BOA', '#GBE', '#CBO', '#ABAY', '#ABSC', '#ABYS', '#DASH', '#ENAT', '#BUNA',
        'CBE', 'BOA', 'GBE', 'CBO', 'ABAY', 'ABSC', 'ABYS', 'DASH', 'ENAT', 'BUNA',
        '#bank', '#Commercial Bank of Ethiopia', '#bankofabyssinia', '#GlobalBankEthiopia', '#BoAEth', '#Coopbank', '#berhanbank'
        'bank', 'Commercial Bank of Ethiopia', 'bankofabyssinia', 'GlobalBankEthiopia', 'BoAEth', 'Coopbank', 'berhanbank'
    ]
    message = row.get('message', '')
    keyword_found = [kw for kw in keywords if kw in message]
    keyword_found_str = ', '.join(keyword_found) if keyword_found else None
    post_id = row.get('id', '')
    post_link = f"https://t.me/tikvahethiopia/{post_id}"
    views = row.get('views', '')
    date = pd.to_datetime(row['date']).date() if 'date' in row else ''
    post_hour = pd.to_datetime(row['date']).time() if 'date' in row else ''
    hour = int(post_hour.hour) if post_hour else 0
    time_of_day = get_time_of_day(hour)
    return [date, post_link, views, post_hour, keyword_found_str, time_of_day]

# Load the CSV data
df = pd.read_csv('../data/filtered_telegram_data.csv')

# Apply the function to the DataFrame
extracted_data_list = df.apply(extract_attributes, axis=1).values.tolist()

# Filter rows that contain "https://t.me/tikvahethiopia/" in the post_link column
filtered_data = [row for row in extracted_data_list if 'https://t.me/tikvahethiopia/' in row[1]]

# Connect to PostgreSQL and create table
db = PostgresConnection(dbname='bank', user='postgres', password='Mati@1993')
db.connect()
db.create_table()

# Insert extracted data into PostgreSQL
db.insert_data(filtered_data)

# Close the connection
db.close_connection()

Connected to PostgreSQL database!
Table 'telegram_posts' created successfully!
Data inserted successfully!
Connection closed.


In [4]:
import psycopg2
from psycopg2 import sql, extras

class PostgresConnection:
    def __init__(self, dbname, user, password, host='localhost', port='5432'):
        self.dbname = dbname
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.conn = None
        self.cursor = None

    def connect(self):
        try:
            self.conn = psycopg2.connect(
                dbname=self.dbname,
                user=self.user,
                password=self.password,
                host=self.host,
                port=self.port
            )
            self.cursor = self.conn.cursor()
            print("Connected to PostgreSQL database!")
        except Exception as e:
            print(f"Error: {e}")

    def create_telegram_posts_table(self):
        create_table_query = """
        CREATE TABLE IF NOT EXISTS telegram_posts (
            id SERIAL PRIMARY KEY,
            keyword TEXT,
            post_link TEXT,
            views INTEGER,
            date DATE,
            post_hour TIME,
            time_of_day TEXT
        );
        """
        try:
            self.cursor.execute(create_table_query)
            self.conn.commit()
            print("Table 'telegram_posts' created successfully!")
        except Exception as e:
            print(f"Error creating table: {e}")
            self.conn.rollback()

    def create_google_play_reviews_table(self):
        create_table_query = """
        CREATE TABLE IF NOT EXISTS google_play_reviews (
            bank TEXT,
            appId TEXT,
            reviewId TEXT,
            userName TEXT,
            userImage TEXT,
            thumbsUpCount INTEGER,
            reviewCreatedVersion TEXT,
            at TIMESTAMP,
            replyContent TEXT,
            repliedAt TIMESTAMP,
            appVersion TEXT,
            score INTEGER,
            content TEXT,
            keywords TEXT,
            LDA_Category TEXT,
            Sentiment TEXT,
            Insight TEXT
        );
        """
        try:
            self.cursor.execute(create_table_query)
            self.conn.commit()
            print("Table 'google_play_reviews' created successfully!")
        except Exception as e:
            print(f"Error creating table: {e}")
            self.conn.rollback()

    def create_google_play_downloads_table(self):
        create_table_query = """
        CREATE TABLE IF NOT EXISTS google_play_downloads (
            bank TEXT,
            appId TEXT,
            date DATE,
            downloads TEXT
        );
        """
        try:
            self.cursor.execute(create_table_query)
            self.conn.commit()
            print("Table 'google_play_downloads' created successfully!")
        except Exception as e:
            print(f"Error creating table: {e}")
            self.conn.rollback()

    def insert_data(self, data, table_name, columns):
        if self.conn is not None:
            try:
                insert_query = sql.SQL("""
                INSERT INTO {} ({}) VALUES %s
                """).format(
                    sql.Identifier(table_name),
                    sql.SQL(', ').join(map(sql.Identifier, columns))
                )
                extras.execute_values(self.cursor, insert_query, data)
                self.conn.commit()
                print(f"Data inserted successfully into {table_name}!")
            except Exception as e:
                print(f"Error inserting data: {e}")
                self.conn.rollback()

    def close_connection(self):
        if self.conn is not None:
            self.cursor.close()
            self.conn.close()
            print("Connection closed.")

In [6]:
import sys
sys.path.append('../src')
import pandas as pd
from connection import PostgresConnection

# Load the CSV data
df_reviews = pd.read_csv('../data/google_play_reviews.csv')
df_downloads = pd.read_csv('../data/google_play_downloads.csv')
df_filtered_telegram_data = pd.read_csv('../data/filtered_telegram_data.csv')

# Ensure column names are lowercase to match the table definitions
df_reviews.columns = [col.lower() for col in df_reviews.columns]
df_downloads.columns = [col.lower() for col in df_downloads.columns]
df_filtered_telegram_data.columns = [col.lower() for col in df_filtered_telegram_data.columns]

# Connect to the PostgreSQL database
db = PostgresConnection(dbname='bank', user='postgres', password='Mati@1993')
db.connect()

# Create tables
db.create_telegram_posts_table()
db.create_google_play_reviews_table()
db.create_google_play_downloads_table()

# Insert data into the tables
db.insert_data(df_reviews.values.tolist(), 'google_play_reviews', df_reviews.columns.tolist())
db.insert_data(df_downloads.values.tolist(), 'google_play_downloads', df_downloads.columns.tolist())
db.insert_data(df_filtered_telegram_data.values.tolist(), 'telegram_posts', df_filtered_telegram_data.columns.tolist())

# Close the database connection
db.close_connection()

Connected to PostgreSQL database!
Table 'telegram_posts' created successfully!
Table 'google_play_reviews' created successfully!
Table 'google_play_downloads' created successfully!
Error inserting data: column "bank" of relation "google_play_reviews" does not exist
LINE 2: ...               INSERT INTO "google_play_reviews" ("bank", "a...
                                                             ^

Error inserting data: column "bank" of relation "google_play_downloads" does not exist
LINE 2: ...             INSERT INTO "google_play_downloads" ("bank", "a...
                                                             ^

Data inserted successfully into telegram_posts!
Connection closed.


In [15]:
# Connect to the database and inspect the table schema
from sqlalchemy import inspect

inspector = inspect(engine)
columns = inspector.get_columns('google_play_reviews')
for column in columns:
    print(column['name'])


bank
appId
reviewId
userName
userImage
thumbsUpCount
reviewCreatedVersion
at
replyContent
repliedAt
appVersion
score
content
keywords
LDA_Category
Sentiment
Insight


In [17]:
import pandas as pd
from sqlalchemy import create_engine, inspect

# Database connection parameters
db_user = 'postgres'
db_password = 'Mati%401993'  # URL-encoded password
db_host = 'localhost'  # Adjust if your database is hosted elsewhere
db_port = '5432'       # Default PostgreSQL port
db_name = 'bank'

# Connection string
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Inspect the table schema
inspector = inspect(engine)
columns = inspector.get_columns('google_play_reviews')
table_columns = [column['name'] for column in columns]
print("Table columns:", table_columns)

# File paths for your CSV files
reviews_csv_path = '../data/google_play_reviews.csv'
downloads_csv_path = '../data/google_play_downloads.csv'
subscribers_csv_path = '../data/boa-telegram-channel-subscriber.csv'

# Reading the CSV files
reviews_df = pd.read_csv(reviews_csv_path, parse_dates=['at', 'repliedAt'])
downloads_df = pd.read_csv(downloads_csv_path, parse_dates=['date'])
subscribers_df = pd.read_csv(subscribers_csv_path, parse_dates=['date'])

# Preprocess the downloads column to remove commas and the '+' sign
downloads_df['downloads'] = downloads_df['downloads'].str.replace(',', '').str.replace('+', '')

# Rename DataFrame columns to match the table schema
rename_dict = {
    'bank': 'bank',
    'appId': 'appId',
    'reviewId': 'reviewId',
    'userName': 'userName',
    'userImage': 'userImage',
    'thumbsUpCount': 'thumbsUpCount',
    'reviewCreatedVersion': 'reviewCreatedVersion',
    'replyContent': 'replyContent',
    'repliedAt': 'repliedAt',
    'appVersion': 'appVersion',
    'score': 'score',
    'content': 'content',
    'keywords': 'keywords',
    'LDA_Category': 'LDA_Category',  # Ensure LDA_Category matches the casing
    'Sentiment': 'Sentiment',
    'Insight': 'Insight'
}

# Apply renaming to reviews DataFrame
reviews_df = reviews_df.rename(columns=rename_dict)

# Ensure no duplicate columns
reviews_df = reviews_df.loc[:, ~reviews_df.columns.duplicated()]

# Keep only the columns present in the table
reviews_df = reviews_df[table_columns]

# Verify columns match the target schema
print("google_play_reviews DataFrame columns:", reviews_df.columns)

# Sending data to PostgreSQL
reviews_df.to_sql('google_play_reviews', engine, if_exists='append', index=False)
downloads_df.to_sql('google_play_downloads', engine, if_exists='append', index=False)
subscribers_df.to_sql('boa_telegram_channel_subscriber', engine, if_exists='append', index=False)

print("Data has been inserted successfully.")

Table columns: ['bank', 'appId', 'reviewId', 'userName', 'userImage', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion', 'score', 'content', 'keywords', 'LDA_Category', 'Sentiment', 'Insight']
google_play_reviews DataFrame columns: Index(['bank', 'appId', 'reviewId', 'userName', 'userImage', 'thumbsUpCount',
       'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion',
       'score', 'content', 'keywords', 'LDA_Category', 'Sentiment', 'Insight'],
      dtype='object')
Data has been inserted successfully.
